<a href="https://colab.research.google.com/github/dedemasutti/ProjectsinPython/blob/master/Python__automatizando_tarefas_de_gest%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install openpyxl
!pip install matplotlib


In [ ]:
import openpyxl
from openpyxl.styles import Font
import matplotlib.pyplot as plt
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def gerar_relatorio_mensal(planilha_entrada, planilha_saida):
    # Carrega a planilha de entrada
    wb = openpyxl.load_workbook(planilha_entrada)
    sheet = wb.active

    # Cria a planilha de saída
    wb_saida = openpyxl.Workbook()
    sheet_saida = wb_saida.active

    # Copia os cabeçalhos da planilha de entrada para a planilha de saída
    for col_num, cell in enumerate(sheet[1], 1):
        sheet_saida.cell(row=1, column=col_num).value = cell.value
        sheet_saida.cell(row=1, column=col_num).font = Font(bold=True)

    # Percorre as linhas da planilha de entrada e realiza cálculos
    total_vendido_por_produto = {}
    for row_num, row in enumerate(sheet.iter_rows(min_row=2), 2):
        # Copia os dados da planilha de entrada para a planilha de saída
        for col_num, cell in enumerate(row, 1):
            sheet_saida.cell(row=row_num, column=col_num).value = cell.value

        # Realiza cálculos e adiciona novas informações na planilha de saída
        quantidade_vendida = row[2].value
        valor_unitario = row[3].value
        total_vendido = quantidade_vendida * valor_unitario

        sheet_saida.cell(row=row_num, column=5).value = total_vendido

        # Atualiza o total vendido por produto
        produto = row[1].value
        if produto in total_vendido_por_produto:
            total_vendido_por_produto[produto] += total_vendido
        else:
            total_vendido_por_produto[produto] = total_vendido

    # Salva a planilha de saída
    wb_saida.save(planilha_saida)

    # Gera gráfico de vendas por produto
    produtos = list(total_vendido_por_produto.keys())
    valores = list(total_vendido_por_produto.values())

    plt.bar(produtos, valores)
    plt.xlabel('Produtos')
    plt.ylabel('Total Vendido')
    plt.title('Vendas por Produto')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig('grafico_vendas.png')

def enviar_relatorio_por_email(remetente, destinatario, assunto, corpo, anexo):
    # Configuração do servidor SMTP
    servidor_smtp = 'smtp.example.com'
    porta_smtp = 587
    usuario_smtp = 'seu_usuario'
    senha_smtp = 'sua_senha'

    # Criação da mensagem de e-mail
    mensagem = MIMEMultipart()
    mensagem['From'] = remetente
    mensagem['To'] = destinatario
    mensagem['Subject'] = assunto

    # Adiciona o corpo da mensagem
    mensagem.attach(MIMEText(corpo, 'plain'))

    # Adiciona o anexo
    with open(anexo, 'rb') as arquivo:
        anexo_mime = MIMEText(arquivo.read(), 'base64', 'png')
        anexo_mime.add_header('Content-Disposition', 'attachment', filename=anexo)
        mensagem.attach(anexo_mime)

    # Envia o e-mail
    with smtplib.SMTP(servidor_smtp, porta_smtp) as servidor:
        servidor.starttls()
        servidor.login(usuario_smtp, senha_smtp)
        servidor.send_message(mensagem)

    print('E-mail enviado com sucesso!')

# Exemplo de uso
planilha_entrada = 'dados_vendas.xlsx'
planilha_saida = 'relatorio_mensal.xlsx'
grafico_vendas = 'grafico_vendas.png'

gerar_relatorio_mensal(planilha_entrada, planilha_saida)
enviar_relatorio_por_email('remetente@example.com', 'destinatario@example.com', 'Relatório Mensal de Vendas', 'Segue o relatório mensal de vendas.', grafico_vendas)
